In [5]:
import pandas as pd
from pandas import json_normalize
import numpy as np
import json as js
import ast as ast
import re
import pyarrow as pa
import pyarrow.parquet as pq

## ETL 

### Carga de datos

Se realizo la carga de datos desde los archivos JSON dados. Para cada DataFrame se creo una copia de seguridad ante de "explotar" los archivos y comenzar las trasformaciones.

In [6]:
games_org = pd.read_json('output_steam_games.json',lines=True)

In [7]:
games = games_org.copy()

In [8]:
list_items = []
archivo2 = r'australian_users_items.json'
with open(archivo2, encoding='utf-8') as file2:
    for line2 in file2.readlines():
        list_items.append(ast.literal_eval(line2))

items_org = pd.DataFrame(list_items)

In [9]:
items = items_org.copy()
items = items.explode('items',ignore_index=True)

In [10]:
list_rev = []
archivo1 = r'australian_user_reviews.json'
with open(archivo1, encoding='utf-8') as file:
    for line in file.readlines():
        list_rev.append(ast.literal_eval(line))

reviews_org = pd.DataFrame(list_rev)

In [11]:
reviews = reviews_org.copy()
reviews = reviews.explode('reviews',ignore_index=True)

### Revision de Games

#### Informacion general

Se revisa el estado de los datos. Se tienen 120.445 registros con 13 columnas. De los cuales todos los tipo *object* con excepcion de dos que son tipo *float*: 'early_access' y 'id'

In [12]:
games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns"


In [13]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 11.9+ MB


#### Normalizacion de columnas

De acuerdo con la consigna se considero que las columnas *'url'*, *'reviews_url'*, *'publisher'* y *'title'* no aportaban informacion de valor y se decidio retirarlas del DataFrame. Tambien se tuvo en cuenta en el caso de *'publisher'* que las consignas iban orientadas a consultas unicamente sobre el *'developer'* y que *'title'* contenia informacion repetivtiva y menor con respecto a *'app_name'*

In [14]:
games.drop(columns=['url','reviews_url','publisher','title'],inplace=True)

Se decidio renombrar las columnas *'id'* y *'app_name'* para que coincidieran con los nombres de las columnas en los DataFrames **Reviews** e **Items** que hacen referencia a la misma informacion

In [15]:
games.rename(columns={'id':'item_id','app_name':'item_name'},inplace=True)

In [16]:
games.head()

,genres,item_name,release_date,tags,specs,price,early_access,item_id,developer
0,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,NaN,NaN,None


#### Columna 'price'

Para realizar un analisis adecuado de esta variable y resolver consultas como el total de dinero gastado por un usuario fue necesario realizar el cambio de tipo de dato de *string* a un valor numerico, en este cado *float*. Para efectuar este cambio fue necesario cambiar primero los valor que estaban en texto como 'Free To Play' y 'Free Movie' a 0.00 y los que estaban establecidos como 'Starting at $449.00' a el valor indicado (en este caso 449.00).

In [17]:
games['price'].unique()

array([None, 4.99, 'Free To Play', 'Free to Play', 0.99, 2.99, 3.99, 9.99,
       18.99, 29.99, 'Free', 10.99, 1.5899999999999999, 14.99, 1.99,
       59.99, 8.99, 6.99, 7.99, 39.99, 19.99, 7.49, 12.99, 5.99, 2.49,
       15.99, 1.25, 24.99, 17.99, 61.99, 3.49, 11.99, 13.99, 'Free Demo',
       'Play for Free!', 34.99, 74.76, 1.49, 32.99, 99.99, 14.95, 69.99,
       16.99, 79.99, 49.99, 5.0, 44.99, 13.98, 29.96, 119.99, 109.99,
       149.99, 771.71, 'Install Now', 21.99, 89.99,
       'Play WARMACHINE: Tactics Demo', 0.98, 139.92, 4.29, 64.99,
       'Free Mod', 54.99, 74.99, 'Install Theme', 0.89, 'Third-party',
       0.5, 'Play Now', 299.99, 1.29, 3.0, 15.0, 5.49, 23.99, 49.0, 20.99,
       10.93, 1.3900000000000001, 'Free HITMAN™ Holiday Pack', 36.99,
       4.49, 2.0, 4.0, 9.0, 234.99, 1.9500000000000002, 1.5, 199.0, 189.0,
       6.66, 27.99, 10.49, 129.99, 179.0, 26.99, 399.99, 31.99, 399.0,
       20.0, 40.0, 3.33, 199.99, 22.99, 320.0, 38.85, 71.7, 59.95, 995.0,
       27.49,

Se cambiaron los valores de texto de la columna 'price' a 0.00 con excepcion de los valores 'Starting at $499.00' y 'Starting at $449.00' que se 
establecieron en los precios del texto ($499 y $449)

In [18]:
games['price']=games['price'].replace('Starting at $499.00',499.00)
games['price']=games['price'].replace('Starting at $449.00',449.00)

In [19]:
mask = games['price'].apply(lambda x: isinstance(x, str))
games.loc[mask, 'price'] = 0.00

In [20]:
games['price']=games['price'].astype(float)

In [21]:
games['price'].unique().size

148

#### Columna 'release_year'

Debido a que en las consultas a responder se hace referecia unicamente al año de lanzamiento se decido crear una nueva columna que contenga solamente el año de lanzamiento para hacer mas eficiente todo el proceso. Para esto se creo una funcion usando la libreria *re* y se borro la columna *'release_date'*. Los valores que no contenian año o estaban en nulo se establecieron en nulo,

In [22]:
#Funcion para extraer el año de la fecha de lanzamiento
def extract_year(value):
    
    match = re.search(r'\b(19|20)\d{2}\b', str(value))
    if match:
        return int(match.group(0)) 
    return None

In [23]:
#Columna solo con el año
games['release_year'] = games['release_date'].apply(extract_year)
games.drop(columns=['release_date'],inplace=True)

#### Version final

A continuacion se presenta el estado final del DataFrame a ser exportado para continuar con la fase de EDA.

In [24]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        28852 non-null  object 
 1   item_name     32133 non-null  object 
 2   tags          31972 non-null  object 
 3   specs         31465 non-null  object 
 4   price         30758 non-null  float64
 5   early_access  32135 non-null  float64
 6   item_id       32133 non-null  float64
 7   developer     28836 non-null  object 
 8   release_year  29964 non-null  float64
dtypes: float64(4), object(5)
memory usage: 8.3+ MB


In [25]:
games.head()

,genres,item_name,tags,specs,price,early_access,item_id,developer,release_year
0,None,None,None,None,NaN,NaN,NaN,None,NaN
1,None,None,None,None,NaN,NaN,NaN,None,NaN
2,None,None,None,None,NaN,NaN,NaN,None,NaN
3,None,None,None,None,NaN,NaN,NaN,None,NaN
4,None,None,None,None,NaN,NaN,NaN,None,NaN


### Revision de Items

#### Informacion general

Se revisa el estado inicial de los datos. Se tienen 5.170.015 registros con 5 columnas. De los cuales la columna *'items'* contiene un diccionario con la informacion del item adquirido por el usuario. Se puede observar que para las columnas *'user_id'*, *'items_count'*, *'steam_id'* y *'user_url'* hay valores duplicados, esto se debe a que para cada item adquirido por el usuario se tiene una fila que contiene nuevamente toda la informacion del usuario.

In [26]:
items

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '10', 'item_name': 'Counter-Strike..."
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '20', 'item_name': 'Team Fortress ..."
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '30', 'item_name': 'Day of Defeat'..."
3,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '40', 'item_name': 'Deathmatch Cla..."
4,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '50', 'item_name': 'Half-Life: Opp..."
...,...,...,...,...,...
5170010,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '373330', 'item_name': 'All Is Dus..."
5170011,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '388490', 'item_name': 'One Way To..."
5170012,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '521570', 'item_name': 'You Have 1..."
5170013,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '519140', 'item_name': 'Minds Eyes..."


Se revisa la composion del diccionario al interior de la columna, para saber que columnas se deben obtener al final del proceso de desanidado. En este caso son 4: *'item_id'*, *'item_name'*, *'playtime_forever'* y *'playtime_2weeks'*.

In [27]:
items.iloc[0,4]

{'item_id': '10',
 'item_name': 'Counter-Strike',
 'playtime_forever': 6,
 'playtime_2weeks': 0}

#### Desanidado de la columna 'items'

Se realiza el desanidado de la columna con la ayuda de la funcion *json_normalize* lo cual crea un nuevo DataFrame el cual se une al que ya estabamos trabajando y se borra la columna *'items'* que contenia la informacion que ahora tenemos en columnas independientes. Se lleno con el valor *None* los datos que eran un diccionario vacio para poder identificar estos valores nulos mas facilmente, 

In [28]:
items_expanded = pd.json_normalize(items['items'])
items_expanded=items_expanded.replace('',None)

In [29]:
items_expanded.head()

,item_id,item_name,playtime_forever,playtime_2weeks
0,10,Counter-Strike,6.0,0.0
1,20,Team Fortress Classic,0.0,0.0
2,30,Day of Defeat,7.0,0.0
3,40,Deathmatch Classic,0.0,0.0
4,50,Half-Life: Opposing Force,0.0,0.0


In [30]:
items = items.join(items_expanded)
items.drop(columns=['items'],inplace=True)

In [31]:
items.head()

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6.0,0.0
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,Team Fortress Classic,0.0,0.0
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30,Day of Defeat,7.0,0.0
3,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,40,Deathmatch Classic,0.0,0.0
4,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,50,Half-Life: Opposing Force,0.0,0.0


#### Normalizacion de columnas

Se reviso el tipo de dato y el contenido de todas las columnas. Se tienen 5 columnas con tipo de dato *object+*, una con valor *int* (*'items_count'*) y 2 con tipo *float* (*'playtime_forever'* y *'playtime_2weeks'*)-

In [32]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5170015 entries, 0 to 5170014
Data columns (total 8 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       int64  
 2   steam_id          object 
 3   user_url          object 
 4   item_id           object 
 5   item_name         object 
 6   playtime_forever  float64
 7   playtime_2weeks   float64
dtypes: float64(2), int64(1), object(5)
memory usage: 315.6+ MB


Se decidio eliminar las siguientes columnas:
- __'user_url'__ ya que no contiene informacion requerida para la resolucion de las consultas solicitadas.
- __'steam_id'__ contiene la misma informacion que la columna *'user_id'*.
- __'items_count'__ contiene informacion repetida que se puede obtener a traves de la funcion values_count()
- __'playtime_2weeks'__ corresponde al tiempo de juego del usuario en las ultimas 2 semanas. Teniendo en cuenta que para el objetivo del analisis es necesario conocer el tiempo total de juego del usuario y que no se conoce la fecha exacta de la extraccion de los datos se considero que este no aporta informacion de valor para el analisis.

In [33]:
items.drop(columns=['user_url','steam_id','items_count','playtime_2weeks'],inplace=True)

Se realizo el cambio de nombre de la columna *'playtime_forever'* a *'playtime'* ya que es un nombre mas explicito sobre la informacion que contiene.

In [34]:
items.rename(columns={'playtime_forever':'playtime'},inplace=True)

In [35]:
items

,user_id,item_id,item_name,playtime
0,76561197970982479,10,Counter-Strike,6.0
1,76561197970982479,20,Team Fortress Classic,0.0
2,76561197970982479,30,Day of Defeat,7.0
3,76561197970982479,40,Deathmatch Classic,0.0
4,76561197970982479,50,Half-Life: Opposing Force,0.0
...,...,...,...,...
5170010,76561198329548331,373330,All Is Dust,0.0
5170011,76561198329548331,388490,One Way To Die: Steam Edition,3.0
5170012,76561198329548331,521570,You Have 10 Seconds 2,4.0
5170013,76561198329548331,519140,Minds Eyes,3.0


#### Cambio tipo de datos - Columna 'item_id'

Se cambio el tipo de dato de esta columna para normalizar el tipo de dato de esta columna que es la que relaciona los 3 DataFrames que se estan trabajando y posteriormente sera necesario conectar.

In [36]:
items['item_id']=items['item_id'].astype(float)

#### Conversion de tiempos - Columna 'playtime_forever'

Se convierten los tiempos de minutos a horas para facilitar el analisis y que se encuentre en las unidades correspondientes a las consultas que se realizaran mas adelante.

In [37]:
items['playtime']=items['playtime']/60

#### Version final

A continuacion se presenta el estado final del DataFrame a ser exportado para continuar con la fase de EDA.

In [38]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5170015 entries, 0 to 5170014
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   user_id    object 
 1   item_id    float64
 2   item_name  object 
 3   playtime   float64
dtypes: float64(2), object(2)
memory usage: 157.8+ MB


In [39]:
items.head()

,user_id,item_id,item_name,playtime
0,76561197970982479,10.0,Counter-Strike,0.100000
1,76561197970982479,20.0,Team Fortress Classic,0.000000
2,76561197970982479,30.0,Day of Defeat,0.116667
3,76561197970982479,40.0,Deathmatch Classic,0.000000
4,76561197970982479,50.0,Half-Life: Opposing Force,0.000000


### Revision de Reviews

#### Informacion general

Se revisa el estado inicial de los datos. Se tienen 59.333 registros con 3 columnas. De los cuales la columna *'reviews'* contiene un diccionario con la informacion de la reseña realizada por el usuario. Se puede observar que para las columnas *'user_id'* y *'user_url'* hay valores duplicados, esto se debe a que para cada item reseñado por el usuario se tiene una fila que contiene nuevamente toda la informacion del usuario.

In [40]:
reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20..."
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011...."
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011..."
3,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014...."
4,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2..."
...,...,...,...
59328,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 10.', 'la..."
59329,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 8.', 'las..."
59330,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '1 person found this review funny', ..."
59331,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '', 'posted': 'Posted July 20.', 'la..."


Se revisa la composion del diccionario al interior de la columna, para saber que columnas se deben obtener al final del proceso de desanidado. En este caso son 7.

In [41]:
reviews.iloc[1,2]

{'funny': '',
 'posted': 'Posted July 15, 2011.',
 'last_edited': '',
 'item_id': '22200',
 'helpful': 'No ratings yet',
 'recommend': True,
 'review': "It's unique and worth a playthrough."}

#### Desanidado de la columna 'reviews'

Se realiza el desanidado de la columna con la ayuda de la funcion *json_normalize* lo cual crea un nuevo DataFrame el cual se une al que ya estabamos trabajando y se borra la columna *'reviews'* que contenia la informacion que ahora tenemos en columnas independientes. Se lleno con el valor *None* los datos que eran un diccionario vacio para poder identificar estos valores nulos mas facilmente.

In [42]:
reviews_expanded = pd.json_normalize(reviews['reviews'])
reviews_expanded=reviews_expanded.replace('',None)

In [43]:
reviews_expanded

,funny,posted,last_edited,item_id,helpful,recommend,review
0,None,"Posted November 5, 2011.",None,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,None,"Posted July 15, 2011.",None,22200,No ratings yet,True,It's unique and worth a playthrough.
2,None,"Posted April 21, 2011.",None,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,None,"Posted June 24, 2014.",None,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,None,"Posted September 8, 2013.",None,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...
59328,None,Posted July 10.,None,70,No ratings yet,True,a must have classic from steam definitely wort...
59329,None,Posted July 8.,None,362890,No ratings yet,True,this game is a perfect remake of the original ...
59330,1 person found this review funny,Posted July 3.,None,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
59331,None,Posted July 20.,None,730,No ratings yet,True,:D


In [44]:
reviews = reviews.join(reviews_expanded)
reviews.drop(columns=['reviews'],inplace=True)

In [45]:
reviews

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,None,"Posted November 5, 2011.",None,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,None,"Posted July 15, 2011.",None,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,None,"Posted April 21, 2011.",None,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,None,"Posted June 24, 2014.",None,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,None,"Posted September 8, 2013.",None,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...,...,...
59328,76561198312638244,http://steamcommunity.com/profiles/76561198312...,None,Posted July 10.,None,70,No ratings yet,True,a must have classic from steam definitely wort...
59329,76561198312638244,http://steamcommunity.com/profiles/76561198312...,None,Posted July 8.,None,362890,No ratings yet,True,this game is a perfect remake of the original ...
59330,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,1 person found this review funny,Posted July 3.,None,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
59331,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,None,Posted July 20.,None,730,No ratings yet,True,:D


#### Normalizacion de columnas

Se reviso el tipo de dato y el contenido de todas las columnas. Se tienen 9 columnas todas de tipo de dato *object*

In [46]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59333 entries, 0 to 59332
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      59333 non-null  object
 1   user_url     59333 non-null  object
 2   funny        8151 non-null   object
 3   posted       59305 non-null  object
 4   last_edited  6140 non-null   object
 5   item_id      59305 non-null  object
 6   helpful      59305 non-null  object
 7   recommend    59305 non-null  object
 8   review       59275 non-null  object
dtypes: object(9)
memory usage: 4.1+ MB


Se borraron las columnas *'user_url'*, *'funny'*, *'posted'*, *'last_edited'* y *'helpful'* ya que no aportaban informacion de valor para el analisis que se realizara mas adelante.

In [47]:
reviews.drop(columns=['user_url','funny','posted','last_edited','helpful'],inplace=True)

In [48]:
reviews

,user_id,item_id,recommend,review
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,22200,True,It's unique and worth a playthrough.
2,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,251610,True,I know what you think when you see this title ...
4,js41637,227300,True,For a simple (it's actually not all that simpl...
...,...,...,...,...
59328,76561198312638244,70,True,a must have classic from steam definitely wort...
59329,76561198312638244,362890,True,this game is a perfect remake of the original ...
59330,LydiaMorley,273110,True,had so much fun plaing this and collecting res...
59331,LydiaMorley,730,True,:D


#### Cambio de tipo de datos - Columnas 'item_id' y 'recommend'

Se cambio el tipo de dato de esta columna para normalizar el tipo de dato de esta columna que es la que relaciona los 3 DataFrames que se estan trabajando y posteriormente sera necesario conectar. En el caso de *'recommend'* se realizó el cambio a *boolean* para facilitar el analisis ya que es mas eficiente de esa manera y representa la misma informacion.

In [49]:
reviews['item_id']=reviews['item_id'].astype(float)

In [50]:
reviews['recommend']=reviews['recommend'].astype(bool)

#### Adicion de item_name al DataFrame

Se decidio agregar la columna *'item_name'* al DataFrame apoyandonos de los demas DataFrames para que al momento de hacer el analisis la informacion presentada sea de mayor valor para la compañia, debido a que se podra visualizar el nombre del juego en lugar de solamente el identificador.

In [51]:
reviews = reviews.merge(games[['item_name','item_id']].notnull(), on='item_id', how='left')

In [52]:
reviews

,user_id,item_id,recommend,review,item_name
0,76561197970982479,1250.0,True,Simple yet with great replayability. In my opi...,NaN
1,76561197970982479,22200.0,True,It's unique and worth a playthrough.,NaN
2,76561197970982479,43110.0,True,Great atmosphere. The gunplay can be a bit chu...,NaN
3,js41637,251610.0,True,I know what you think when you see this title ...,NaN
4,js41637,227300.0,True,For a simple (it's actually not all that simpl...,NaN
...,...,...,...,...,...
59328,76561198312638244,70.0,True,a must have classic from steam definitely wort...,NaN
59329,76561198312638244,362890.0,True,this game is a perfect remake of the original ...,NaN
59330,LydiaMorley,273110.0,True,had so much fun plaing this and collecting res...,NaN
59331,LydiaMorley,730.0,True,:D,NaN


In [53]:
#lista de los item_id unicos en la tabla items
items_unicos = items.drop(items[items.duplicated(subset='item_id')].index)

In [54]:
items_unicos

,user_id,item_id,item_name,playtime
0,76561197970982479,10.0,Counter-Strike,0.100000
1,76561197970982479,20.0,Team Fortress Classic,0.000000
2,76561197970982479,30.0,Day of Defeat,0.116667
3,76561197970982479,40.0,Deathmatch Classic,0.000000
4,76561197970982479,50.0,Half-Life: Opposing Force,0.000000
...,...,...,...,...
4926140,76561198081168593,375450.0,NOBUNAGA'S AMBITION: Souzou SengokuRisshiden,0.850000
5108879,76561198101994484,353390.0,Alienware Steam Machine,0.000000
5121271,76561198107283457,354280.0,ChaosTower,2.733333
5135856,inven,433920.0,Aveyond 4: Shadow Of The Mist,0.000000


In [55]:
reviews = reviews.merge(items_unicos[['item_id','item_name']], on='item_id', how='left')

In [56]:
reviews

,user_id,item_id,recommend,review,item_name_x,item_name_y
0,76561197970982479,1250.0,True,Simple yet with great replayability. In my opi...,NaN,Killing Floor
1,76561197970982479,22200.0,True,It's unique and worth a playthrough.,NaN,Zeno Clash
2,76561197970982479,43110.0,True,Great atmosphere. The gunplay can be a bit chu...,NaN,Metro 2033
3,js41637,251610.0,True,I know what you think when you see this title ...,NaN,Barbie™ Dreamhouse Party™
4,js41637,227300.0,True,For a simple (it's actually not all that simpl...,NaN,Euro Truck Simulator 2
...,...,...,...,...,...,...
59328,76561198312638244,70.0,True,a must have classic from steam definitely wort...,NaN,Half-Life
59329,76561198312638244,362890.0,True,this game is a perfect remake of the original ...,NaN,Black Mesa
59330,LydiaMorley,273110.0,True,had so much fun plaing this and collecting res...,NaN,Counter-Strike Nexon: Zombies
59331,LydiaMorley,730.0,True,:D,NaN,Counter-Strike: Global Offensive


In [57]:
reviews['item_name'] = reviews['item_name_x'].combine_first(reviews['item_name_y'])

In [58]:
reviews.drop(columns=['item_name_x','item_name_y'],inplace=True)

In [59]:
reviews

,user_id,item_id,recommend,review,item_name
0,76561197970982479,1250.0,True,Simple yet with great replayability. In my opi...,Killing Floor
1,76561197970982479,22200.0,True,It's unique and worth a playthrough.,Zeno Clash
2,76561197970982479,43110.0,True,Great atmosphere. The gunplay can be a bit chu...,Metro 2033
3,js41637,251610.0,True,I know what you think when you see this title ...,Barbie™ Dreamhouse Party™
4,js41637,227300.0,True,For a simple (it's actually not all that simpl...,Euro Truck Simulator 2
...,...,...,...,...,...
59328,76561198312638244,70.0,True,a must have classic from steam definitely wort...,Half-Life
59329,76561198312638244,362890.0,True,this game is a perfect remake of the original ...,Black Mesa
59330,LydiaMorley,273110.0,True,had so much fun plaing this and collecting res...,Counter-Strike Nexon: Zombies
59331,LydiaMorley,730.0,True,:D,Counter-Strike: Global Offensive


#### Version final

A continuacion se presenta el estado final del DataFrame a ser exportado para continuar con la fase de EDA.

In [60]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59333 entries, 0 to 59332
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    59333 non-null  object
 1   item_id    59305 non-null  object
 2   recommend  59333 non-null  bool  
 3   review     59275 non-null  object
 4   item_name  52781 non-null  object
dtypes: bool(1), object(4)
memory usage: 1.9+ MB


In [61]:
reviews

,user_id,item_id,recommend,review,item_name
0,76561197970982479,1250.0,True,Simple yet with great replayability. In my opi...,Killing Floor
1,76561197970982479,22200.0,True,It's unique and worth a playthrough.,Zeno Clash
2,76561197970982479,43110.0,True,Great atmosphere. The gunplay can be a bit chu...,Metro 2033
3,js41637,251610.0,True,I know what you think when you see this title ...,Barbie™ Dreamhouse Party™
4,js41637,227300.0,True,For a simple (it's actually not all that simpl...,Euro Truck Simulator 2
...,...,...,...,...,...
59328,76561198312638244,70.0,True,a must have classic from steam definitely wort...,Half-Life
59329,76561198312638244,362890.0,True,this game is a perfect remake of the original ...,Black Mesa
59330,LydiaMorley,273110.0,True,had so much fun plaing this and collecting res...,Counter-Strike Nexon: Zombies
59331,LydiaMorley,730.0,True,:D,Counter-Strike: Global Offensive


### Exportar DataFrames finales

Se exportan los archivos que se usaran para realizar el Analisis Exploratorio de Datos.

In [62]:
pq.write_table(pa.Table.from_pandas(games),'Archivos ETL/games_etl.parquet')

In [63]:
pq.write_table(pa.Table.from_pandas(items),'Archivos ETL/items_etl.parquet')

In [64]:
pq.write_table(pa.Table.from_pandas(reviews),'Archivos ETL/reviews_etl.parquet')